In [1]:
using Random
using CSV, DataFrames
using PyPlot
using StatsBase

In [2]:
#   db = CSV.read("data.csv")

In [3]:
N = 100 #   ノミネートタイトル数
M = 10^4   #   投票者数
L = 30  #   投票者一人当たりのタイトル経験数スケール(目安)
K = 10  #   一人当たりの投票権利数
LOGSCALE_RANGE = 5

5

In [7]:
#   データ生成のセル①

function make_nominated_title_db(N, value_vector, popularity_vector)
    info_dic = Dict(
        "id"                        => [string(i) for i = 1:N],
        "popularity"                => popularity_vector,
        "experienced counter"       => [0 for _ = 1:N],
        "value"                     => value_vector,
        "simple vote"               => [0 for _ = 1:N],
        "simple vote ranking"       => [0 for _ = 1:N],
        "corrected vote"            => [0 for _ = 1:N],
        "corrected vote ranking"    => [0 for _ = 1:N],
        "professional vote point"   => [0 for _ = 1:N],
        "professional ranking"      => [0 for _ = 1:N],
        "professional vote point 2" => [0 for _ = 1:N],
        "professional ranking 2"    => [0 for _ = 1:N],
    )
    nominated_title_db = DataFrame(info_dic)
    return nominated_title_db
end

function get_index(arr, value)
    for i = 1:length(arr)
        if arr[i] == value
            return i
        end
    end
end


get_index (generic function with 1 method)

In [39]:
#   データ生成のセル②

function make_data(N, M, L, K, nominated_title_db)
    info_dic = Dict(
        "name"                      => [string(i) for i = 1:N],
        "experienced title set"     => [Set() for _ = 1:N],
        "number of experienced title"=> [0 for _ = 1:N],
        "voted title info list"     => [[] for _ = 1:N],
        "poll"                      => [0 for _ = 1:N],
        "waight"                    => [0.0 for _ = 1:N],
    )

    voter_db = DataFrame(info_dic)
    col = names(voter_db)
    j1 = get_index(col, "experienced title set")
    j2 = get_index(col, "number of experienced title")
    waight_col_j = get_index(col, "waight")
    voted_title_col_j = get_index(col, "voted title info list")
    poll_col_j = get_index(col, "poll")

    popularity_arr, value_arr = nothing, nothing
    popularity_arr = nominated_title_db[:,"popularity"]
    value_arr = nominated_title_db[:,"value"]

    for i = 1:size(voter_db)[1]
        t = sample([i for i = 1:N], Weights(popularity_arr/sum(popularity_arr)), K)
        voter_db[i, j1] = Set(t)
        voter_db[i, j2] = length(Set(t))
        voter_db[i, waight_col_j] = length(Set(t)) #   ここはどうする？
        kh = [0 for _ = 1:maximum(t)]
        for h = 1:length(t)
            kh[t[h]] += 1
        end
        for h = 1:length(kh)
            if kh[h] != 0
                push!(voter_db[i, voted_title_col_j], (h, kh[h]))
            end
        end
        voter_db[i, poll_col_j] = length(Set(t))
    end
    for i = 1:size(voter_db)[1]
        voter_db[i,waight_col_j] = voter_db[i,waight_col_j] .* K/voter_db[i,poll_col_j]  #   票数が少なかったら重み補正　ここはないほうが良い場合もあるかな
    end
    voter_db[:,waight_col_j] /= sum(voter_db[:,waight_col_j])
    return voter_db
end

popularity_vector = [exp(-LOGSCALE_RANGE*i/N) for i = 0:N-1]
value_vector = randn(N) + [(N-i)/N for i = 0:N-1]
value_vector += [minimum(value_vector) for _ = 1:length(value_vector)]
value_vector /= maximum(value_vector)
nominated_title_db = make_nominated_title_db(N, value_vector, popularity_vector)

voter_db = make_data(N, M, L, K, nominated_title_db)
println(voter_db)

100×6 DataFrame
 Row │ experienced title set              name    number of experienced title  poll   voted title info list              waight  
     │ Set{Any}                           String  Int64                        Int64  Vector{Any}                        Float64 
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Set(Any[5, 32, 4, 7, 25, 33, 95,…  1                                10     10  Any[(4, 1), (5, 1), (7, 1), (8, …     0.01
   2 │ Set(Any[92, 41, 6, 11, 7, 26, 8,…  2                                 9      9  Any[(3, 1), (6, 2), (7, 1), (8, …     0.01
   3 │ Set(Any[5, 50, 94, 33, 13, 40, 2…  3                                10     10  Any[(1, 1), (5, 1), (12, 1), (13…     0.01
   4 │ Set(Any[39, 35, 21, 9, 12, 8, 17…  4                                10     10  Any[(1, 1), (3, 1), (8, 1), (9, …     0.01
   5 │ Set(Any[5, 34, 15, 2, 10, 9, 26,…  5                                 9 

In [23]:
info_dic = Dict(
    "name"                      => [string(i) for i = 1:N],
    "experienced title list"    => [[] for _ = 1:N],
    "number of experienced title"=> [0 for _ = 1:N],
    "voted title info list"     => [[] for _ = 1:N],
    "poll"                      => [1 for _ = 1:N],
    "waight"                    => [0.0 for _ = 1:N],
)
voter_db = DataFrame(info_dic)
col = names(voter_db)
voter_db

,experienced title list,name,number of experienced title,poll,voted title info list
,Array…,String,Int64,Int64,Array…
1,[],1,0,1,[]
2,[],2,0,1,[]
3,[],3,0,1,[]
4,[],4,0,1,[]
5,[],5,0,1,[]
6,[],6,0,1,[]
7,[],7,0,1,[]
8,[],8,0,1,[]
9,[],9,0,1,[]
